In [25]:
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
from tqdm import tqdm 
import random
import pandas as pd

In [47]:
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [27]:
random.seed(42)

In [19]:
texts = []
labels = []

with open('domain1_train.json', 'r') as file:
    for line in file:
        # Parse each line as a JSON object
        data = json.loads(line)
        # Append the text and label to their respective lists
        texts.append(data["text"])
        labels.append(data["label"])

In [20]:
SentenceLength = []
for i in texts:
    SentenceLength.append(len(i))

maxSequence = max(SentenceLength)
print(maxSequence)

238


In [21]:
X = np.array(texts, dtype=object)
y = np.array(labels)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
model = Sequential()
#input_dim:一共5000种tokens; input_length:目前句子最大长度为238
#后期需要通过CV来确认output_dim的最佳数值
model.add(Embedding(input_dim = 5001, output_dim = 175, input_length = maxSequence))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
X_train_padded_sequences = pad_sequences(X_train, maxlen = maxSequence, padding='post', value = 5000)
X_test_padded_sequences = pad_sequences(X_test, maxlen = maxSequence, padding='post', value = 5000)


X_train = tf.convert_to_tensor(X_train_padded_sequences, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test_padded_sequences, dtype=tf.float32)


In [26]:
model.fit(X_train, y_train, epochs=10, batch_size=100, validation_data=(X_test, y_test))

Epoch 1/10
156/156 [==============================] - 70s 438ms/step - loss: 0.6934 - accuracy: 0.5001 - val_loss: 0.6933 - val_accuracy: 0.5018
Epoch 2/10
156/156 [==============================] - 70s 449ms/step - loss: 0.6934 - accuracy: 0.5017 - val_loss: 0.6940 - val_accuracy: 0.4982
Epoch 3/10
156/156 [==============================] - 77s 492ms/step - loss: 0.6934 - accuracy: 0.4997 - val_loss: 0.6931 - val_accuracy: 0.5018
Epoch 4/10
156/156 [==============================] - 83s 536ms/step - loss: 0.6932 - accuracy: 0.4958 - val_loss: 0.6931 - val_accuracy: 0.4982
Epoch 5/10
156/156 [==============================] - 84s 538ms/step - loss: 0.6932 - accuracy: 0.4906 - val_loss: 0.6931 - val_accuracy: 0.5018
Epoch 6/10
 33/156 [=====>........................] - ETA: 1:07 - loss: 0.6932 - accuracy: 0.5021

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.0.1+cu117
True


In [3]:
# Load data
texts = []
labels = []

with open('domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        texts.append(data["text"])
        labels.append(data["label"])

# Determine max sequence length
max_sequence = max(len(seq) for seq in texts)

# Convert data to numpy arrays
X = np.array(texts, dtype=object)
y = np.array(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # Get the final output of the LSTM
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [4]:
# Initialize the model
vocab_size = 5000  # Adjust as needed
embedding_dim = 150
hidden_dim = 128
model = TextClassificationModel(vocab_size, embedding_dim, hidden_dim)

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [5]:
# Convert data to PyTorch tensors
X_train_padded_sequences = [torch.tensor(seq) for seq in X_train]
X_train_padded_sequences = pad_sequence(X_train_padded_sequences, batch_first=True, padding_value=0)
X_test_padded_sequences = [torch.tensor(seq) for seq in X_test]
X_test_padded_sequences = pad_sequence(X_test_padded_sequences, batch_first=True, padding_value=0)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [6]:
model = model.cuda()
X_train_padded_sequences = X_train_padded_sequences.cuda()
X_test_padded_sequences = X_test_padded_sequences.cuda()
y_train = y_train.cuda()
y_test = y_test.cuda()

In [7]:
# Training loop
epochs = 10
batch_size = 10

for epoch in range(epochs):
    for i in range(0, len(X_train_padded_sequences), batch_size):
        optimizer.zero_grad()
        batch_X = X_train_padded_sequences[i:i + batch_size]
        batch_y = y_train[i:i + batch_size]
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y.unsqueeze(1))
        loss.backward()
        optimizer.step()

In [8]:
# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_padded_sequences)
    y_pred = (y_pred > 0.5).float()  # Convert to binary predictions
    accuracy = (y_pred == y_test.unsqueeze(1)).float().mean()

print(f'Test accuracy: {accuracy:.4f}')

OutOfMemoryError: CUDA out of memory. Tried to allocate 7.82 GiB (GPU 0; 2.00 GiB total capacity; 933.58 MiB already allocated; 654.18 MiB free; 958.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

hidden_dim = 128， embedding_dim = 150 准确率为0.4982

In [45]:
#SVM


texts = []
labels = []

# Assuming the file 'domain1_train.json' contains data like the examples provided.
with open('domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        texts.append(data["text"])
        labels.append(data["label"])

with open('domain2_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        texts.append(data["text"])
        labels.append(data["label"])


# Determine max sequence length
max_sequence = max(len(seq) for seq in texts)

# Pad sequences to max_sequence with zeros
texts_padded = [seq + [-1]*(max_sequence - len(seq)) for seq in texts]

# Convert data to numpy arrays
X = np.array(texts_padded)
y = np.array(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
# # Initialize the SVM classifier
# clf = SVC(kernel='linear')

# # Train the SVM classifier
# clf.fit(X_train, y_train)

# # Predict on the test set
# y_pred = clf.predict(X_test)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy * 100:.2f}%")

CNN

In [114]:
# Load data
texts = []
labels = []

with open('domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        texts.append(data["text"])
        labels.append(data["label"])

# data_with_label_1 = [pair for pair in zip(texts[:9750], labels[:9750]) if pair[1] == 1]
# data_with_label_0 = [pair for pair in zip(texts[-9750:], labels[-9750:]) if pair[1] == 0]

# selected_data_label_1 = random.sample(data_with_label_1, 2150)
# selected_data_label_0 = random.sample(data_with_label_0, 2150)

# selected_data = selected_data_label_1 + selected_data_label_0
# texts, labels = zip(*selected_data)
# texts = list(texts)
# labels = list(labels)

with open('domain2_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        texts.append(data["text"])
        labels.append(data["label"])

# Determine max sequence length
max_sequence = max(len(seq) for seq in texts)

# Convert data to numpy arrays
X = np.array(texts, dtype=object)
y = np.array(labels)

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [116]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim, output_dim, dropout):
        super(TextCNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * num_filters, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.output = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim=1))
        dense = self.fc(cat)
        return self.output(dense)

In [117]:
vocab_size = 5000  # Adjust as needed
embedding_dim = 100
num_filters = 75
filter_sizes = [3, 4, 5]
hidden_dim = 128
output_dim = 1
dropout = 0.5

In [118]:
model = TextCNN(vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim, output_dim, dropout)

In [119]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

In [103]:
X_train_padded_sequences = [torch.tensor(seq) for seq in X_train]
X_train_padded_sequences = pad_sequence(X_train_padded_sequences, batch_first=True, padding_value=0)
X_test_padded_sequences = [torch.tensor(seq) for seq in X_test]
X_test_padded_sequences = pad_sequence(X_test_padded_sequences, batch_first=True, padding_value=0)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [104]:
epochs = 100
batch_size = 64


for epoch in range(epochs):
    model.train()
    num_batches = len(X_train_padded_sequences) // batch_size
    with tqdm(total=num_batches, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
        for i in range(0, len(X_train_padded_sequences), batch_size):
            optimizer.zero_grad()
            batch_X = X_train_padded_sequences[i:i + batch_size]
            batch_y = y_train[i:i + batch_size]
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            pbar.update(1)

Epoch 1/40:   0%|          | 0/430 [00:00<?, ?batch/s]

Epoch 40/40: 100%|██████████| 430/430 [08:38<00:00,  1.21s/batch]


In [105]:
# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_padded_sequences)
    y_pred = (y_pred > 0.5).float()  # Convert to binary predictions
    accuracy = (y_pred == y_test.unsqueeze(1)).float().mean()

print(f'Test accuracy: {accuracy:.4f}')

Test accuracy: 0.8577


In [106]:
test_data = []
with open('test_set.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        test_data.append(data)

In [107]:
def preprocess_test_data(data):
    # Extract text and id from the data
    texts = [data_item["text"] for data_item in data]
    ids = [data_item["id"] for data_item in data]
    
    # Convert text to PyTorch tensors and pad sequences
    text_tensors = [torch.tensor(seq) for seq in texts]
    padded_text = pad_sequence(text_tensors, batch_first=True, padding_value=0)
    
    return ids, padded_text

In [108]:
test_ids, X_test_padded_sequences = preprocess_test_data(test_data)

In [109]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_padded_sequences)
    y_pred = (y_pred > 0.5).int() 

In [110]:
y_pred = y_pred.squeeze(1).tolist()

In [111]:
results_df = pd.DataFrame({'id': test_ids, 'class': y_pred})

In [112]:
results_df.to_csv('predictions5.csv', index=False)

In [113]:
torch.save(model.state_dict(), 'model_76.2.pth')